# Tutorial: Die `Equation` Klasse
Source: [PINA-Equation](https://mathlab.github.io/PINA/_rst/tutorials/tutorial12/tutorial.html) <br>

## Definitionen

## Problemdefinition
<strong>Annahme:</strong>
$$
\begin{cases}
  \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} &=v\frac{\partial^2 u}{\partial x^2} \quad &\text{mit} \quad x \in (0,1) \quad t>0\\
  u(x,0) &=-\sin(\pi x)\\
  u(x,t) &=0 \quad &\text{mit} \quad x=\pm1 \\ \\
  \text{mit} \quad v &=\frac{0.01}{\pi}
\end{cases}
$$

In [ ]:
#useful imports
from pina.problem import SpatialProblem, TimeDependentProblem
from pina.equation import Equation, FixedValue, FixedGradient, FixedFlux
from pina.geometry import CartesianDomain
import torch
from pina.operators import grad, laplacian
from pina import Condition

In [ ]:
class Burgers1D(TimeDependentProblem, SpatialProblem):

    # define the burger equation
    def burger_equation(input_, output_):
        du = grad(output_, input_)
        ddu = grad(du, input_, components=['dudx'])
        return (
            du.extract(['dudt']) +
            output_.extract(['u'])*du.extract(['dudx']) -
            (0.01/torch.pi)*ddu.extract(['ddudxdx'])
        )

    # define initial condition
    def initial_condition(input_, output_):
        u_expected = -torch.sin(torch.pi*input_.extract(['x']))
        return output_.extract(['u']) - u_expected

    # assign output/ spatial and temporal variables
    output_variables = ['u']
    spatial_domain = CartesianDomain({'x': [-1, 1]})
    temporal_domain = CartesianDomain({'t': [0, 1]})

    # problem condition statement
    conditions = {
        'gamma1': Condition(location=CartesianDomain({'x': -1, 't': [0, 1]}), equation=FixedValue(0.)),
        'gamma2': Condition(location=CartesianDomain({'x':  1, 't': [0, 1]}), equation=FixedValue(0.)),
        't0': Condition(location=CartesianDomain({'x': [-1, 1], 't': 0}), equation=Equation(initial_condition)),
        'D': Condition(location=CartesianDomain({'x': [-1, 1], 't': [0, 1]}), equation=Equation(burger_equation)),
    }